Notebook for doing my data analysis for my Thesis on E. Coli mutation data from Experimental Evolution experiments sourced from AleDb

# Set up

Running packages

In [ ]:
import glob
import pandas as pd
import os
import google.colab
import numpy as np
import matplotlib.pyplot as plt
from Bio.SubsMat import MatrixInfo
import scipy
from sklearn import decomposition
import seaborn

Uploading data from local to google collab server

In [ ]:
from google.colab import files
uploaded = files.upload()

# Making Dataframes

Collecting data for DataFrame from converted files on desktop

In [ ]:
df= pd.DataFrame()

In [ ]:
experimentid = 1
for file in uploaded.keys():
    data = pd.read_excel(file, "Sheet1", header =4).assign(ExperimentID = experimentid)
    df = df.append(data, ignore_index=True)
    experimentid +=1


In [ ]:
df

In [ ]:
# df.dtypes

Check types of mutations to look for new types that need to be added to the conversion script.

In [ ]:
# dftest = df[df["TYPE"] == "CON"]
# print(dftest)

In [ ]:
# df.groupby("TYPE").count()

Checking Chrom for different versions

In [ ]:
df.groupby("CHROM").count()

Adding column to experiments for grouping them on severity of condition: 1 is severe, 2 is mild

In [ ]:
condition = [2,2,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,1,2,2,1,1,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1]
df["Severity"] = ""
for i,row in df.iterrows():
  df.at[i, "Severity"] = condition[(df.iloc[i]["ExperimentID"])-1]

# Some basic visualizations

Distribution of mutations for all experiments, do not need title.

In [ ]:
df["TYPE"].value_counts().plot(kind = 'bar')
print(df["TYPE"].value_counts())

For our analysis we are going to group all the Amplifications together

In [ ]:
df = df.replace(to_replace= "AMP2", value = "AMP")
df = df.replace(to_replace= "AMP3", value = "AMP")
df = df.replace(to_replace= "AMP4", value = "AMP")
df = df.replace(to_replace= "AMP5", value = "AMP")

We now remove CON and INV since there are so few they do not show up (15 and 3 respectively)

In [ ]:
df = df[df.TYPE != "CON"]
df = df.reset_index(drop=True)
df = df[df.TYPE != "INV"]
df = df.reset_index(drop=True)
df = df[df.TYPE != "REPL"]
df = df.reset_index(drop=True)
print(df["TYPE"].unique())

In [ ]:
total = df["TYPE"].value_counts().rename_axis('unique_values').reset_index(name='counts')
total_valu =total['counts'].sum()
total["Rel_Dist"] = ""
for i,row in total.iterrows():
  total.at[i, "Rel_Dist"] = total.loc[i]["counts"] / total_valu
total

In [ ]:
total = df["TYPE"].value_counts().rename_axis('unique_values').reset_index(name='counts')
total_valu =total['counts'].sum()
# Make some labels.
labels = []
total["Rel_Dist"] = ""
for i,row in total.iterrows():
  total.at[i, "Rel_Dist"] = total.loc[i]["counts"] / total_valu
  labels.append(total.loc[i]["counts"])
total.set_index("unique_values",drop=False,inplace=True)
print(labels)
colors = {"SNP": '#41D3BD', "DEL": '#E63462', "INS": '#666A86', "MNV": '#C7EFCF', "AMP": '#EEF5DB'}
types = list(colors.keys())
ax = total['Rel_Dist'].plot(kind='bar', width = 1.0, x = 'unique_values', y = 'Rel_Dist', color=[colors[i] for i in total['unique_values']])
# handles = [plt.Rectangle((0,0),1,1, color=colors[type]) for type in types]
# plt.legend(handles, types, fontsize = 20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
ax.set_xlabel("")
ax.set_ylim(0,1)
ax.set_ylabel("Frequency", fontsize = 20)

We can check this distribution for every experiment as well.

In [ ]:
Experiment_dataframes_all = []
for experiment in df["ExperimentID"].unique():
  Experiment_dataframes_all.append(df[df["ExperimentID"] ==experiment])
pca_mutation_type_df = pd.DataFrame(columns = ["SNP", "DEL", "INS", "MNV", "AMP"])

In [ ]:
pca_mutation_type_df

In [ ]:
fig = plt.figure()
fig.set_figheight(30)
fig.set_figwidth(30)
# fig.suptitle('Types of Mutations per Experiment', fontsize=16)

for experiment, num in zip(df["ExperimentID"].unique(), range(1,55)):
  ax = fig.add_subplot(11,5,num)
  Experiment_data = df[df["ExperimentID"] ==experiment]
  total = Experiment_data["TYPE"].value_counts().rename_axis('unique_values').reset_index(name='counts')
  total_valu =total['counts'].sum()
  total["Rel_Dist"] = ""
  values = [0,0,0,0,0]
  for i,row in total.iterrows():
    total.at[i, "Rel_Dist"] = total.loc[i]["counts"] / total_valu
    if total.loc[i]["unique_values"] == "SNP":
      values[0]= (total.loc[i]["Rel_Dist"])
    elif total.loc[i]["unique_values"] == "DEL":
      values[1] = total.loc[i]["Rel_Dist"]
    elif total.loc[i]["unique_values"] == "INS":
      values[2] = total.loc[i]["Rel_Dist"]
    elif total.loc[i]["unique_values"] == "MNV":
      values[3] = total.loc[i]["Rel_Dist"]
    elif total.loc[i]["unique_values"] == "AMP":
      values[4] = total.loc[i]["Rel_Dist"]
  a_series = pd. Series(values, index = pca_mutation_type_df. columns)
  pca_mutation_type_df = pca_mutation_type_df. append(a_series, ignore_index=True)  
  colors = {"SNP": '#41D3BD', "DEL": '#E63462', "INS": '#666A86', "MNV": '#C7EFCF', "AMP": '#EEF5DB'}
  types = list(colors.keys())
  bx = total['Rel_Dist'].plot(kind='bar', width = 1.0, sharey=True, x = 'unique_values', y = 'Rel_Dist', color=[colors[i] for i in total['unique_values']])
  # handles = [plt.Rectangle((0,0),1,1, color=colors[type]) for type in types]
  # plt.legend(handles, types)
  plt.xticks([])
  bx.set_xlabel("")
  bx.set_ylim(0,1)
  ax.set_title(experiment)
# types = list(colors.keys())
# handles = [plt.Rectangle((0,0),1,1, color=colors[type]) for type in types]
# fig.legend(handles, types)
fig.tight_layout()

Looking at distributions through PCA to look for interesting different experiments. Make a dataframe with 6 columns? One for each type of mutation with the relative percent in it and do pca on this?

In [ ]:
pca_mutation_type_df

In [ ]:
pca = decomposition.PCA(n_components=2)
pca.fit(pca_mutation_type_df)
x = pca.transform(pca_mutation_type_df)
colors = []
for i,row in pca_mutation_type_df.iterrows():
  colors.append(condition[i-1])
plt.scatter(x[:,0], x[:,1], s=40, c = colors, cmap = 'coolwarm')
plt.xticks([])
plt.yticks([])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

Look at types of mutations in a heatmap

In [ ]:
index = list(pca_mutation_type_df.index.values)
seaborn.heatmap(pca_mutation_type_df, cmap='coolwarm')
plt.ylabel("Experiment")

In [ ]:
df.hist(column = 'TIME', bins = 20)

Want to get maximum experiment length in flask for each experiment to see experiment length comparisons

A large number of the experiments only recorded one flask so we filter them out since it is an indefinite time

In [ ]:
experiment_lengths = []
experiments_with_one = []
experiments_greaterthan200 = []
experiment_min_values = []
counter = 1
while counter < 55:
  experiment_df = df.loc[df["ExperimentID"] == counter]
  experiment_lengths.append(experiment_df["TIME"].max())
  experiment_min_values.append(experiment_df["TIME"].min())
  if experiment_df["TIME"].max() == 1:
    experiments_with_one.append(counter)
  if experiment_df["TIME"].max() > 200:
    experiments_greaterthan200.append(counter)
  counter +=1

# print(experiment_lengths)
# print(experiments_with_one)
# print(experiments_greaterthan200)
# print(experiment_min_values)

Making new dataframe that tracks experiment ID, and max and min length for experiment by flask counts

In [ ]:
Experiment_df = pd.DataFrame({"ExperimentID": list(range(1,55)), "Min_Flask": experiment_min_values, "Max_Flask": experiment_lengths})
# print(Experiment_df)

Now we can do a graph showing the experiment length comparisons using flask number as a representation of time. Since a few of our experiments have an insanely high experiment length we need to group them and do a above x count to make the plot work better. We can try the count now at above 200 or could do even lower counts.

In [ ]:
for i in range(len(experiment_lengths)):
  if experiment_lengths[i] >= 200:
    experiment_lengths[i] = 200
print(experiment_lengths)   

In [ ]:
plt.hist(experiment_lengths, bins = 20)
plt.ylabel("Number of Experiments")
plt.xlabel("Number of Flasks")
# plt.title("Experiment Length in Flasks")

Now we trim our experiment list to remove experiments that only report 1 flask time, remove experiments that have a max value of 1 flask. Too difficult to do this as a dataframe because of the weird indexing so we will make it a dictionary.

In [ ]:
for i,row in Experiment_df.iterrows():
  if row["Min_Flask"] == row["Max_Flask"]:
    Experiment_df = Experiment_df.drop(i)
Experiment_df = Experiment_df.reset_index(drop=True)
Experiment_dict = {}
for i,row in Experiment_df.iterrows():
  Experiment_dict[Experiment_df.iloc[i]["ExperimentID"]] = [Experiment_df.iloc[i]["Min_Flask"], Experiment_df.iloc[i]["Max_Flask"]]
# print(Experiment_dict)
# Experiment_df

Next step is to create a new column in our dataframe we we use normalized flask values so 1 would be close to 0 and max value would be 1 so we can see how mutations accumulate over time. Need to make new dataframe with mutations that meet the above requirements.

In [ ]:
IDs_tokeep = Experiment_df["ExperimentID"].values
Normalization_df = df.loc[df["ExperimentID"].isin(IDs_tokeep)]
Normalization_df = Normalization_df.reset_index(drop=True)
Normalization_df


Here we create a new column for normalized time that gives the time point in the experiment. If there is 10 flasks in an experiment than at time 1 we know its 10% through the experiment.

In [ ]:
Normalization_df["Normalized_Time"] = ""
for i,row in Normalization_df.iterrows():
  experimentid = Normalization_df.iloc[i]["ExperimentID"]
  min_flask = Experiment_dict.get(experimentid)[0]
  max_flask = Experiment_dict.get(experimentid)[1]
  Normalization_df.at[i, "Normalized_Time"] = (Normalization_df.iloc[i]["TIME"] - min_flask) / (max_flask - min_flask)

In [ ]:
Normalization_df

In [ ]:
yticks =Normalization_df["ExperimentID"].unique()

Now that we have added a normalized value we can look at a density plot for mutations, like how many mutations are there occuring throughout an experiment. Do more mutations occur at the start or end?

In [ ]:
times = Normalization_df["Normalized_Time"].values
weights = np.ones_like(times) / len(times)
plt.hist(times, bins = 20, weights = weights)
plt.ylabel("Percent of Mutations")
plt.xlabel("Relative time in Experiment")
newdf = pd.DataFrame(columns = ["0-20", "20-40", "40-60", "60-80", "80-100"])



Try doing this plot per experiment for comparisons. We start by making a list of the dataframes for each experiment.

In [ ]:
Experiment_dataframes = []
for experiment in Experiment_df["ExperimentID"].unique():
  Experiment_dataframes.append(Normalization_df[Normalization_df["ExperimentID"] ==experiment])

In [ ]:
fig = plt.figure()
fig.set_figheight(25)
fig.set_figwidth(25)
# fig.suptitle('Mutation accumulation per experiment', fontsize=16)
for experiment, num in zip(Experiment_df["ExperimentID"].unique(), range(1,30)):
  ax = fig.add_subplot(5,6,num)
  Experiment_data = Normalization_df[Normalization_df["ExperimentID"] ==experiment]
  times = Experiment_data["Normalized_Time"].values
  weights = np.ones_like(times) / len(times)
  plt.hist(times, bins = 20, weights = weights)
  ax.set_ylim([0,.8])
  # plt.hist(Experiment_data["Normalized_Time"], bins = 20, density = True)
  ax.set_title(experiment)
  # plt.ylabel("Number of Mutations")
  # plt.xlabel("Relative time in Experiment")
  # plt.title("Mutation occurrence throughout experiment")
  zeroto20 = 0
  twentyoneto40= 0
  fortyoneto60 = 0
  sixty1to80 = 0
  eighty1to100 = 0

  for i in times:
    if i <= .20:
      zeroto20 +=1
    elif .20 < i <= .40:
      twentyoneto40 +=1
    elif .40 < i <= .60:
      fortyoneto60 +=1
    elif .60 < i < .80:
      sixty1to80 +=1
    else:
      eighty1to100 +=1
  newdf = newdf.append({"0-20": zeroto20/len(times), "20-40": twentyoneto40/len(times), "40-60": fortyoneto60/len(times), "60-80": sixty1to80/len(times), "80-100": eighty1to100/len(times)}, ignore_index=True)
fig.tight_layout

In [ ]:
seaborn.set(rc={'figure.figsize':(11.7,8.27)})
chart = seaborn.heatmap(newdf, cmap='coolwarm', yticklabels=yticks)
chart.set_yticklabels(chart.get_yticklabels(),rotation=0)
plt.ylabel("Experiment")
plt.xlabel("Relative time through Experiment")

Here we are going to look at distribution of intergenic vs non intergenic mutations. Intergenic mutations are represented as NA in the Gene column so we can find the distribution by looking for non NA values in the GEN column.

In [ ]:
Total_mutations = df.shape[0]
Intergenic = df["GEN"].isna().sum()
Non_intergenic = Total_mutations - Intergenic
Intergenic_perc = Intergenic/Total_mutations
Non_perc = Non_intergenic/Total_mutations
Intergenicdf = pd.DataFrame({'Type':['Intergenic', 'Non-intergenic'], 'Count':[Intergenic_perc, Non_perc]})
ax = Intergenicdf.plot.bar(x='Type', y='Count', rot=0, legend = False)
ax.set_xlabel("")
# plt.title("Distribution of Mutations")

To do this easier we can assign a column to each experiment having a 0 if intergenic and a 1 if not

In [ ]:
df["Intergenic_or_not"] = ""
for i,row in df.iterrows():
  if pd.isna(df.iloc[i]["GEN"]):
    df.at[i, "Intergenic_or_not"] = 'Intergenic'
  else:
    df.at[i, "Intergenic_or_not"] = 'Non-intergenic'
df["Intergenic_or_not"] = df.Intergenic_or_not.astype('category')
Intergenic_df_scatter = pd.DataFrame(columns = ["Intergenic_perc", "Non-intergenic_perc"])

Here we can check each experiment individually.

In [ ]:
fig = plt.figure()
fig.set_figheight(30)
fig.set_figwidth(30)
# fig.suptitle('Location of mutations per Experiment', fontsize=16)
for experiment, num in zip(df["ExperimentID"].unique(), range(1,55)): 
  ax = fig.add_subplot(8,7,num)
  values = []
  Experiment_data = df[df["ExperimentID"] ==experiment]
  total = Experiment_data["Intergenic_or_not"].value_counts().rename_axis('unique_values').reset_index(name='counts')
  total_valu =total['counts'].sum()
  total["Rel_Dist"] = ""
  total.at[0, "Rel_Dist"] = total.loc[0]["counts"] / total_valu
  total.at[1, "Rel_Dist"] = total.loc[1]["counts"] / total_valu
  values.append(total.loc[0]["Rel_Dist"])
  values.append(total.loc[1]["Rel_Dist"])
  colors = {"Non-intergenic": 'r', "Intergenic": 'b'}
  types = list(colors.keys())
  ax = total['Rel_Dist'].plot(kind='bar', sharey=True, width = .5, x = 'unique_values', y = 'Rel_Dist', color=[colors[i] for i in total['unique_values']])
  handles = [plt.Rectangle((0,0),1,1, color=colors[type]) for type in types]
  plt.xticks([])
  ax.set_xlabel("")
  ax.set_ylim(0,1)
  ax.set_title(experiment)
  a_series = pd. Series(values, index = Intergenic_df_scatter. columns)
  Intergenic_df_scatter = Intergenic_df_scatter. append(a_series, ignore_index=True)
# fig.legend(handles, types)
fig.tight_layout()

Now we want to check if some genes are more likely to be targets of mutations than others.First step is to remove NaNs, then to make mutations with multiple genes separated by a comma added.

We can make a scatter plot of these values too to see in a different way how experiments are different.

In [ ]:
Intergenic_df_scatter

In [ ]:
colors = []
for i,row in pca_mutation_type_df.iterrows():
  colors.append(condition[i-1])
plt.scatter(Intergenic_df_scatter["Intergenic_perc"], Intergenic_df_scatter["Non-intergenic_perc"], c = colors, cmap = 'coolwarm')
plt.xlabel("Percent Non-intergenic")
plt.ylabel("Percent Intergenic")

In [ ]:
genes = []
Non_intergenic_mutations = df[df['GEN'].notna()]
Non_intergenic_mutations = Non_intergenic_mutations.reset_index(drop=True)
for i, row in Non_intergenic_mutations.iterrows():
  if "," in Non_intergenic_mutations.iloc[i]["GEN"]:
    gene_list = Non_intergenic_mutations.iloc[i]["GEN"].split(",")
    for i in gene_list:
        genes.append(i)
  else:
    genes.append(Non_intergenic_mutations.iloc[i]["GEN"])

Only include genes that appear more than one time.

---



In [ ]:
pd.Series(genes).value_counts()[lambda x : x>200].plot(kind = 'bar')
#  figsize = (30,10)
plt.ylabel("Count")

Next step is to use a blossum matrix to get a score for SNPs and the amino acid they change to. Can do two different approaches. First is look at all mutations with amino acid information in df and just plot the scores. Second is use normalization_df and look at scores relative to mutation time to see if more drastic changes happen at the start or end of the experiment. I will use the blossum 50 matrix for scoring.

In [ ]:
Aminoacid_df = df[df['∆AA'].notna()]
Normalized_amino_df = Normalization_df[Normalization_df["∆AA"].notna()]
Aminoacid_df = Aminoacid_df.reset_index(drop=True)
Normalized_amino_df = Normalized_amino_df.reset_index(drop=True)

In [ ]:
Aminoacid_df

Need to make nested dictionary for scoring values. Using Biopython Blosum50 matrix dictionary

In [ ]:
# MatrixInfo.blosum50

In [ ]:
Aminoacid_df["AA1"] = ""
Aminoacid_df["AA2"] = ""
Aminoacid_df["Score"] = ""
for i,row in Aminoacid_df.iterrows():
  AA1 = Aminoacid_df.iloc[i]["∆AA"][0].upper()
  AA2 = Aminoacid_df.iloc[i]["∆AA"][-1].upper()
  Aminoacid_df.at[i, "AA1"] = AA1
  Aminoacid_df.at[i, "AA2"] = AA2
Normalized_amino_df["AA1"] = ""
Normalized_amino_df["AA2"] = ""
for i,row in Normalized_amino_df.iterrows():
  AA1 = Normalized_amino_df.iloc[i]["∆AA"][0].upper()
  AA2 = Normalized_amino_df.iloc[i]["∆AA"][-1].upper()
  Normalized_amino_df.at[i, "AA1"] = AA1
  Normalized_amino_df.at[i, "AA2"] = AA2

In [ ]:
# Aminoacid_df

In [ ]:
Aminoacid_df = Aminoacid_df[Aminoacid_df['AA1'] != '*']
Aminoacid_df = Aminoacid_df[Aminoacid_df['AA2'] != '*']
Aminoacid_df = Aminoacid_df.reset_index(drop=True)
# Aminoacid_df

In [ ]:
Normalized_amino_df = Normalized_amino_df[Normalized_amino_df.AA1 != '*']
Normalized_amino_df = Normalized_amino_df[Normalized_amino_df.AA2 != '*']

Normalized_amino_df = Normalized_amino_df.reset_index(drop=True)

for i,row in Aminoacid_df.iterrows():
  AA1 = Aminoacid_df.iloc[i]["∆AA"][0].upper()
  AA2 = Aminoacid_df.iloc[i]["∆AA"][-1].upper()
  try:
    Aminoacid_df.at[i, "Score"] = MatrixInfo.blosum50[AA1, AA2]
  except:
    Aminoacid_df.at[i, "Score"] = MatrixInfo.blosum50[AA2, AA1]
for i,row in Normalized_amino_df.iterrows():
  AA1 = Normalized_amino_df.iloc[i]["∆AA"][0].upper()
  AA2 = Normalized_amino_df.iloc[i]["∆AA"][-1].upper()
  try:
    Normalized_amino_df.at[i, "Score"] = MatrixInfo.blosum50[AA1,AA2]
  except:
    Normalized_amino_df.at[i, "Score"] = MatrixInfo.blosum50[AA2,AA1]


In [ ]:
# Aminoacid_df

Now we can graph this for all experiments and per experiment

In [ ]:
plt.hist(Aminoacid_df["Score"], bins = 20)
plt.ylabel("Number of Mutations")
plt.xlabel("Conservation Score")
plt.axvline(x=0,color='black',linestyle='--')


In [ ]:
fig = plt.figure()
fig.set_figheight(25)
fig.set_figwidth(25)
fig.suptitle('Conservation of Amino Acid Substitutions', fontsize=16)
for experiment, num in zip(df["ExperimentID"].unique(), range(1,55)):
  ax = fig.add_subplot(8,7,num)
  Experiment_data = Aminoacid_df[Aminoacid_df["ExperimentID"] ==experiment]
  plt.hist(Experiment_data["Score"], bins = 20)
  ax.set_title(experiment)
  plt.axvline(x=0,color='black',linestyle='--')
  # plt.ylabel("Number of Mutations")
  # plt.xlabel("Relative time in Experiment")
  # plt.title("Mutation occurrence throughout experiment")

Now we can look at the normalized time dataframe to see conservation scores throughout the experiment. One way to do this is to split our data into two dataframes, one with mutations less than 50% through the experiment and the other greater.

In [ ]:
First_half = Normalized_amino_df[Normalized_amino_df['Normalized_Time'] <= .50]
Second_half = Normalized_amino_df[Normalized_amino_df['Normalized_Time'] > .50]
First_half = First_half.reset_index(drop=True)
Second_half = Second_half.reset_index(drop=True)

In [ ]:
plt.subplot(1,2,1)
plt.hist(First_half["Score"], bins = 20)
plt.ylabel("Number of Mutations")
plt.xlabel("Conservation Score")
plt.axvline(x=0,color='black',linestyle='--')

# plt.title("Conservation of Amino Acid Substitutions")

ax = plt.subplot(1,2,2)
plt.hist(Second_half["Score"], bins = 20)
# plt.ylabel("Number of Mutations")
plt.xlabel("Conservation Score")
ax.set_ylim(0,700)
plt.yticks([])
plt.axvline(x=0,color='black',linestyle='--')

# plt.title("Conservation of Amino Acid Substitutions")

In [ ]:
# plt.hist(Second_half["Score"], bins = 20)
# plt.ylabel("Number of Mutations")
# plt.xlabel("Conservation Score")
# plt.title("Conservation of Amino Acid Substitutions")

We can also do this comparison for each experiment. But I can wait for Brams response and ask about this approach.

# Looking at CAMEL database experiments

Breakdown of CAMEL database and WGS, 136 experiments using WGS out of 632 experiments.

In [ ]:
WGS = 136
Not = 632-136
Total_count = 632
WGS_perc = WGS/Total_count
Not_perc = Not/Total_count
WGSdf = pd.DataFrame({'Type':['WGS', 'Not WGS'], 'Count':[WGS_perc, Not_perc]})
ax = WGSdf.plot.bar(x='Type', y='Count', rot=0, legend = False)
ax.set_xlabel("")
plt.ylabel("Frequency", fontsize = 20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
# plt.title("Distribution of Mutations")

WGS methods used in CAMEL

In [ ]:
WGSdf_tech = pd.DataFrame({'Technology':['Illumina', 'Hybridization array', 'Sanger', '454', 'Pacbio', 'NimbleGen', 'SOLiD', 'Ion Torrent'], 'Count':[114, 11, 9, 4, 3, 3, 2, 2]})

colors = {'Illumina':'#41D3BD', 'Hybridization array':'#E63462', 'Sanger': '#666A86', '454': '#C7EFCF', "Pacbio": "#EEF5DB", "NimbleGen": '#067BC2', "SOLiD": '#84BCDA', "Ion Torrent": '#FFFC31'}         
ax = WGSdf_tech.plot.bar(x='Technology', y='Count', rot=0, color=[colors[i] for i in WGSdf_tech['Technology']], width = 1)
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
plt.legend(handles, labels, fontsize = 15)
plt.xticks([])
ax.set_xlabel("")
plt.yticks(fontsize = 15)
plt.ylabel("Count", fontsize = 20)


Upload files for next parts

In [ ]:
uploaded2 = files.upload()

First graph will be on number of lines in Experiment. Upload json results from API with number of lines per experiment

In [ ]:
numlines = pd.read_json("/content/numberoflines.json")

In [ ]:
numlines = numlines["values"]
numlines

In [ ]:
linesdataframe = pd.DataFrame(columns = ["Number of lines", "Count"])
for i in numlines:
  linesdataframe = linesdataframe.append({"Number of lines": i["value"], "Count": i["number"]}, ignore_index=True)
linesdataframe
linevalues = []
for i, row in linesdataframe.iterrows():
  linevalues += [row["Number of lines"]] * row["Count"]
#Make values greater than 200 = to 200 then have them say greater than 200.
for idx, item in enumerate(linevalues):
  if item > 200:
    linevalues[idx] = 201
plt.hist(linevalues)
plt.ylabel("Experiments", fontsize = 20)
plt.xlabel("Number of Lines", fontsize = 20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)

Second is Experiment Length in days


In [ ]:
numdays = pd.read_json("/content/numberofdays.json")
numdays=numdays["values"]
daysdataframe = pd.DataFrame(columns = ["Number of Days", "Count"])
for i in numdays:
  daysdataframe = daysdataframe.append({"Number of Days": i["value"], "Count": i["number"]}, ignore_index=True)
daysdataframe
dayvalues = []
for i, row in daysdataframe.iterrows():
  dayvalues += [row["Number of Days"]] * row["Count"]
#Make values greater than 200 = to 200 then have them say greater than 200.
for idx, item in enumerate(dayvalues):
  if item > 200:
    dayvalues[idx] = 201
plt.hist(dayvalues)
plt.ylabel("Experiments", fontsize = 20)
plt.xlabel("Number of Days", fontsize =20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)

Third is Number of generations

In [ ]:
numgens = pd.read_json("/content/generations.json")
numgens=numgens["values"]
gendataframe = pd.DataFrame(columns = ["Number of Generations", "Count"])
for i in numgens:
  gendataframe = gendataframe.append({"Number of Generations": i["value"], "Count": i["number"]}, ignore_index=True)
genvalues = []
for i, row in gendataframe.iterrows():
  genvalues += [row["Number of Generations"]] * int(row["Count"])
#Make values greater than 200 = to 200 then have them say greater than 200.
for idx, item in enumerate(genvalues):
  if item > 2000:
    genvalues[idx] = 2001
plt.hist(genvalues)
plt.ylabel("Experiments", fontsize = 20)
plt.xlabel("Number of Generations", fontsize = 20)
plt.xticks(fontsize = 15, rotation = 90)
plt.yticks(fontsize = 15)

WGS per year

In [ ]:
wgsperyear = pd.read_csv("/content/WGSperyearcount")
wgsvalues = pd.DataFrame(columns = ["Year", "Count"])
for i, row in wgsperyear.iterrows():
  val = row[0]
  wgsvalues = wgsvalues.append({"Year": int(val.split("\t")[0]), "Count": int(val.split("\t")[1])}, ignore_index=True)
wgsvalues
ax = wgsvalues.plot.bar(x = "Year", legend = False)
ax.set_ylabel("WGS Experiments", fontsize = 20)
ax.set_xlabel("Year", fontsize = 20)
plt.xticks(fontsize = 15, rotation = 90)
plt.yticks(fontsize = 15)

Experiments per year

In [ ]:
years = pd.read_json("/content/referenceyears.json")
years=years["year"]
ax = years.value_counts().plot.bar()
plt.gca().invert_xaxis()
plt.ylabel("Experiments", fontsize = 20)
plt.xlabel("Year", fontsize = 20)
plt.xticks(fontsize = 15, rotation = 90)
plt.yticks(fontsize = 15)
plt.setp(ax.get_xticklabels()[::2], visible=False)

Recreating species graph to match same format

In [ ]:
species = ["E. coli", "S. cerevisiae", "P. aeruginosa", "P. fluorescens", "Phage phi2", "S. aureus", "Tobacco etch virus", "R. solanacearum", "Other(78 species)"]
values = [112, 41, 29, 20, 10, 7, 6, 5, 115]

speciesdf = pd.DataFrame({'Species': species, 'Count':values})

ax = speciesdf.plot.barh(x='Species', y='Count', rot= 90, legend = False)
plt.xlabel("Experiments", fontsize = 20)
plt.ylabel("")
plt.xticks(fontsize = 15, rotation = 0)
plt.yticks(fontsize = 15, rotation = 0)
plt.gca().invert_yaxis()

Uploading new files and looking at new graphs from adapated tools. Run time took a long time so only 31 of 54 experiments were finished though this still contains all of the largest experiments and the majority of the data.

In [ ]:
uploadedfinal = files.upload()

In [ ]:
finaldf = pd.DataFrame()

In [ ]:
experimentid = 1
for file in uploadedfinal.keys():
    data = pd.read_excel(file, "Sheet1").assign(ExperimentID = experimentid)
    finaldf = finaldf.append(data, ignore_index=True)
    experimentid +=1

The first thing we will look at is the distribution of cellular locations

In [ ]:
finaldf

In [ ]:
# finaldf["Location"].value_counts().plot.bar()
# plt.ylabel("Mutations")
# plt.xticks(rotation = 0)

Looking at impact scores of mutations

In [ ]:
finaldf["impact"].value_counts().plot.bar()
print(finaldf["impact"].value_counts())
plt.ylabel("Mutations")
plt.xticks(rotation = 0)

We can also check which of the 6 categories mutations are affected by PTMs and linear motifs, Stability, Interfaces, Conservation, TFBS, and Start Stop Codons

We can also look at whether mutations affect interactions with other molecules. The higher the score the more likely these interactions are impacted.

In [ ]:
finaldf["Total Interaction Score"] = pd.to_numeric(finaldf["Total Interaction Score"])
plt.hist(finaldf["Total Interaction Score"])
plt.ylabel("Mutations")
plt.xlabel("Mechismo Score")
# finaldf["Total Interaction Score"].hist()